In [1]:
from dask.distributed import Client
import os

import dask

If you have an existing Dask cluster running already, set the scheduler address below. Otherwise, leave it to `None` and a local cluster will be created.

In [2]:
scheduler_address = None #"tcp://10.2.168.161:8786"

if scheduler_address is None:
    from dask_cuda import LocalCUDACluster
    cluster = LocalCUDACluster(n_workers = 2)
    c = Client(cluster)
else:
    c = Client(scheduler_address)
c

/raid/cjnolet/cuml5/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:43799 Dashboard: http://127.0.0.1:37869/status,Cluster Workers: 2 Cores: 2 Memory: 540.95 GB


In [3]:
from nccl_example import get_address

In [4]:
from dask.distributed import wait
from dask.distributed import default_client
import random
import ucp

In [5]:
class CommsBase:
    
    def __init__(self, comms_p2p = False):
        self.client = default_client()

        self.worker_addresses = self.get_workers_()

    def get_workers_(self, parse_address = True):
        """
        Return the list of workers parsed as [(address, port)]
        """
        return list(self.client.has_what().keys())
    
    @staticmethod 
    def func_init_ucp_py(r):
        print("Initiailizing ucp PY")
        ucp.init()
        
        ucp_worker = ucp.get_ucp_worker()
                    
        print("WORKER POINTER (Python): " + hex(ucp_worker))


        """
        *************************
        This is failing!
        """
        get_address(ucp_worker)

        """
        *************************
        """


        
    def init_ucp(self):
        """
        Use ucx-py to initialize ucp endpoints so that every 
        worker can communicate, point-to-point, with every other worker
        """
        
        f = [self.client.submit(CommsBase.func_init_ucp_py, random.random(), workers = [w])
                for w in self.worker_addresses]
        
        wait(f)
        

In [6]:
class Dask_UCX_Demo(CommsBase):
    
    def __init__(self, client):
        super(Dask_UCX_Demo, self).__init__(comms_p2p = True)
        self.client = client
        
        self.init_(1)
        
    def init_(self, verbose = 0):
        self.init_ucp()

In [7]:
demo = Dask_UCX_Demo(c)